
# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur agrégé au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>


# TD11 : Classification bayésienne naïve pour filtrer les spams

### Description

Dans ce TD, nous construirons un filtre anti-spam pour un autre employé de la compagnie Enron. On utilisera autant de mot que nécessaire pour obtenir un classement satisfaisant.

### Données

Dans ce TD, nous utiliserons les messages électroniques d'un autre employé de la compagnie Enron. Veuillez récupérer l'archive *Enron2.zip* sur Moodle et la décompresser dans le répertoire courant du calepin. 

# Chargement des librairies

In [1]:
using MLBase

# Chargement des données

Le code de cette section permet de traiter les fichiers textes correspondant à tous les messages électroniques de l'utilisateurs. Les messages électroniques se trouvent dans les dossier *ham/* et *spam/* de l'utilisateur *Enron2*.

In [2]:
# Récupération des noms de fichier de tous les hams
filesdir = "enron2/ham/"
filename_ham = filesdir.*readdir(filesdir);

In [3]:
# Récupération des noms de fichier de tous les spams
filesdir = "enron2/spam/"
filename_spam = filesdir.*readdir(filesdir);

### Partitionnement des données en ensemble d'entraînement et de validation

Le 2/3 des données constituent l'ensemble d'entraînement et le 1/3 restant l'ensemble de validation.

In [4]:
# Partitionnement des courriels
ham_train = sample(filename_ham, round(Int, 2/3*length(filename_ham)), replace=false, ordered=true)
ham_valid = setdiff(filename_ham, ham_train);

In [5]:
# Partitionnement des pourriels
spam_train = sample(filename_spam, round(Int, 2/3*length(filename_spam)), replace=false, ordered=true)
spam_valid = setdiff(filename_spam, spam_train);

In [72]:
# Vecteur des solutions de l'ensemble de validation (0 = pourriel, 1 = courriel)
Z = vcat(ones(Int64, length(ham_valid)), zeros(Int64, length(spam_valid)));

In [7]:
println("L'échantillon d'entraînement est composé de $(length(ham_train)) courriels et $(length(spam_train)) pourriels.")
println("L'échantillon de validation est composé de $(length(ham_valid)) courriels et $(length(spam_valid)) pourriels.")

L'échantillon d'entraînement est composé de 2907 courriels et 997 pourriels.
L'échantillon de validation est composé de 1454 courriels et 499 pourriels.


# Extraction des occurrences des mots

### Fonctions permettants le traitement des fichiers textes pour la classifications bayésienne naïve.

In [8]:
"""
    wordlisting(filename::String)

Extrait la liste des mots contenus dans le fichier texte `filename`.

### Détails
Ne dénombre pas le nombre d'occurrence des mots dans un fichier. N'est pas sensible aux majuscules ni aux minuscules.
"""
function wordlisting(filename::String)
    
    f = read(filename, String)
    text = replace(f, r"[0123456789]" => "")
    words = split(text, r"\W+")
    filter!(x -> length(x) > 1, words)
    wordlist = unique(words)
    
end

"""
    wordcounting(wordlist::Vector{<:AbstractString})

Dénombre les occurrences des mots dans la liste `wordlist`. 

### Détails
Retourne un dictionnaire ayant comme clé le mot, et la valeur l'occurrence du mot.
"""
function wordcounting(wordlist::Vector{<:AbstractString})

    wordcounts = Dict{String,Int64}()

    for word in wordlist
        wordcounts[word]=get(wordcounts, word, 0) + 1
    end
    
    return wordcounts
    
end

wordcounting

### Extraction de l'occurrence des mots des courriels

In [9]:
ham_wordlist = wordlisting.(ham_train)
ham_wordcounts = wordcounting(vcat(ham_wordlist...))

Dict{String, Int64} with 20734 entries:
  "chem"          => 1
  "piecemeal"     => 1
  "mh"            => 2
  "basisnw"       => 2
  "meyn"          => 2
  "henry"         => 13
  "panellist"     => 1
  "bidder"        => 5
  "hampshire"     => 1
  "connell"       => 5
  "rhonda"        => 1
  "il"            => 14
  "bottomfeeding" => 1
  "gathered"      => 4
  "okazji"        => 2
  "cosimo"        => 2
  "rainstorms"    => 1
  "underground"   => 2
  "backdrop"      => 4
  "angelova"      => 4
  "budgeted"      => 6
  "november"      => 111
  "estabilish"    => 1
  "stress"        => 16
  "backup"        => 4
  ⋮               => ⋮

### Extraction de l'occurrence des mots des pourriels

In [10]:
spam_wordlist = wordlisting.(spam_train)
spam_wordcounts = wordcounting(vcat(spam_wordlist...))

Dict{String, Int64} with 16872 entries:
  "irreplaceable"   => 1
  "alternativetry"  => 1
  "mh"              => 1
  "attache"         => 1
  "henry"           => 1
  "sakarsucks"      => 1
  "polyimide"       => 1
  "vincentsamoasan" => 1
  "bidder"          => 1
  "hampshire"       => 3
  "graand"          => 3
  "il"              => 11
  "msnbc"           => 5
  "archiv"          => 1
  "gathered"        => 12
  "ochoa"           => 1
  "lovers"          => 2
  "realrhapsody"    => 1
  "sobre"           => 3
  "canal"           => 2
  "backup"          => 5
  "caught"          => 3
  "stress"          => 6
  "november"        => 6
  "craftsmanship"   => 1
  ⋮                 => ⋮

# Exercice 1

Considérez le premier mot discriminant comme variable explicative pour classer les messages électroniques en courriels et pourriels :

$$ X_1 = \begin{cases}
0 & \mbox{ si le mot est absent du message ;}\\
1 & \mbox{ si le mot est présent dans le message.}
\end{cases}$$

Utilisez les lois a priori vagues.

### (a) Calculez la probabilité marginale qu'un nouveau message soit un courriel

Autrement dit, calculez la probabilité suivante

$$p_1 = \mathbb{P}(\tilde{Y}=1).$$

In [12]:
n₁ = length(ham_train)
n₀ = length(spam_train)
n = n₁ + n₀

p₁ = n₁ / n

0.7446209016393442

### (b) Calculez la probabilité marginale qu'un nouveau message soit un pourriel

Autrement dit, calculez la probabilité suivante

$$p_0 = \mathbb{P}(\tilde{Y}=0).$$

In [26]:
p₀ = n₀ / n

0.25537909836065575

### (c) Calculez la probabilité prédictive que le mot *click* soit présent dans un pourriel.

Autrement dit, calculez la quantité suivante :

$$p_{01} = \mathbb{P}(\tilde{X}_1=1|\tilde{Y}=0).$$

In [19]:
α₀₁ = 1
β₀₁ = 1
n₀₁ = spam_wordcounts["click"];

In [20]:
p₀₁ = (α₀₁ + n₀₁) / (α₀₁ + β₀₁ + n₀)

0.2182182182182182

### (d) Calculez la probabilité prédictive que le mot *click* soit présent dans un courriel.

Autrement dit, calculez la quantité suivante :

$$p_{11} = \mathbb{P}(\tilde{X}_1=1|\tilde{Y}=1).$$

In [23]:
α₁₁ = 1
β₁₁ = 1
n₁₁ = ham_wordcounts["click"];

In [24]:
p₁₁ = (α₁₁ + n₁₁) / (α₁₁ + β₁₁ + n₁)

0.03196974905465796

### (e) Calculez la probabilité que le message soit un courriel si le mot *click* est présent.

Autrement dit, calculez la probabilité suivante :

$$\mathbb{P}(\tilde{Y}=1|\tilde{X}_1=1) = \frac{\mathbb{P}(\tilde{X}_1=1|\tilde{Y}=1) \times \mathbb{P}(\tilde{Y}=1)}{ \mathbb{P}(\tilde{X}_1=1|\tilde{Y}=0) \times \mathbb{P}(\tilde{Y}=0) + \mathbb{P}(\tilde{X}_1=1|\tilde{Y}=1) \times \mathbb{P}(\tilde{Y}=1)}$$

Avec la notation introduite précédemment, on a que

$$\mathbb{P}(\tilde{Y}=1|\tilde{X}_1=1) = \frac{p_{11} \times p_1}{ p_{01} \times p_0 + p_{11} \times p_1}.$$

In [27]:
p₁₁ * p₁ / (p₀₁ * p₀ + p₁₁ * p₁)

0.2993113462910328

### (f) Calculez la probabilité que le message soit un courriel si le mot *click* est absent.

Autrement dit, calculez la probabilité suivante :

$$\mathbb{P}(\tilde{Y}=1|\tilde{X}_1=0) = \frac{\mathbb{P}(\tilde{X}_1=0|\tilde{Y}=1) \times \mathbb{P}(\tilde{Y}=1)}{ \mathbb{P}(\tilde{X}_1=0|\tilde{Y}=0) \times \mathbb{P}(\tilde{Y}=0) + \mathbb{P}(\tilde{X}_1=0|\tilde{Y}=1) \times \mathbb{P}(\tilde{Y}=1)}$$

Avec la notation introduite précédemment, on a que

$$\mathbb{P}(\tilde{Y}=1|\tilde{X}_1=0) = \frac{(1-p_{11}) \times p_1}{ (1-p_{01}) \times p_0 + (1-p_{11}) \times p_1}.$$

In [28]:
(1 - p₁₁) * p₁ / ((1 - p₀₁) * p₀ + (1 - p₁₁) * p₁)

0.7830982733002408

### (g) Filtrez les messages de l'ensemble de validation

In [75]:
word1 = "click"
Ẑ = Int64[]

for filename in ham_valid
      
    wordlist = wordlisting(filename)
    x̃ = any(wordlist .== word1)
    
    if x̃
        push!(Ẑ, 0)
    else
        push!(Ẑ, 1)
    end
 
end

for filename in spam_valid
      
    wordlist = wordlisting(filename)
    x̃ = any(wordlist .== word1)
    
    if x̃
        push!(Ẑ, 0)
    else
        push!(Ẑ, 1)
    end
 
end

r = roc(Z, Ẑ)

ROCNums{Int64}
  p = 1454
  n = 499
  tp = 1400
  tn = 141
  fp = 358
  fn = 54


# Exercice 2

Voici deux fonctions permettant de calculer les facteurs devant les probabilités marginales pour inclure l'effet de la variable explicative. On va reprendre l'Exercice 1 en utilisant ces fonctions. On pourra par la suite facilement généraliser à plusieurs variables explicatives.

In [33]:
"""
    wordspamliness(word::AbstractString)

Calcul la probabilite que `word` soit présent dans les spams (p₀₁).

### Détails
Le nombre de spams `n₀` et le dénombrement de l'occurrence des mots dans les spams `spam_wordcounts` doivent
être dans le global scope.
"""
function wordspamliness(word::AbstractString)
    
    if haskey(spam_wordcounts, word)
        n₀₁ = spam_wordcounts[word]
    else
        n₀₁ = 0
    end

    p₀₁ = (1 + n₀₁)/(2 + n₀)
    
end

"""
    wordhamliness(word::AbstractString)

Calcul la probabilite que `word` soit présent dans les hams (p₁₁).

### Détails
Le nombre de hams `n₁` et le dénombrement de l'occurrence des mots dans les hams `ham_wordcounts` doivent
être dans le global scope.
"""
function wordhamliness(word::AbstractString)
    
    if haskey(ham_wordcounts, word)
        n₁₁ = ham_wordcounts[word]
    else
        n₁₁ = 0
    end

    p₁₁ = (1 + n₁₁)/(2 + n₁)
    
end

wordhamliness

### (a) Calculez p₀₁ et p₁₁ avec les fonctions données dans la cellules précédentes pour le mot *click*.

In [34]:
p₀₁ = wordspamliness("click")

0.2182182182182182

In [35]:
p₁₁ = wordhamliness("click")

0.03196974905465796

### (b) Écrivez une fonction permettant de filter les courriels en fonction d'un mot

Vous pouvez utilisez la transformation log pour stabiliser numériquement vos calculs

In [67]:
"""
    function filteremail(filename::AbstractString, word::AbstractString)

Filtre le message contenu dans le fichier `filename` en fonction du mot `word`.

### Détail
La valeur 1 est renvoyée si la probabilité prédictive que ce soit un courriel est supérieur à .5. 
Sinon, la valeur 0 est renvoyée
"""
function filteremail(filename::AbstractString, word::AbstractString)
    wordlist = wordlisting(filename)
    
    q₀ = log(p₀)
    q₁ = log(p₁)
    
    if word in wordlist
        q₀ += log(wordspamliness(word))
        q₁ += log(wordhamliness(word))
    else
        q₀ += log(1 - wordspamliness(word))
        q₁ += log(1 - wordhamliness(word))
    end
    
    if q₁ > q₀
        return 1
    else
        return 0
    end
end

filteremail

### (c) Filtrez les messages de l'ensemble de validation avec votre fonction en utilisant le mot *click*.

In [68]:
Z̃ = Int64[]

for filename in vcat(ham_valid, spam_valid)
    push!(Z̃, filteremail(filename, "click"))
end
    
r = roc(Z, Ẑ)

ROCNums{Int64}
  p = 1454
  n = 499
  tp = 1400
  tn = 141
  fp = 358
  fn = 54


# Exercice 3 - Utilisation d'une liste de mots

La cellule suivante permet de trouver les mots les plus discriminants pour classifier les messages entre pourriels et courriels. On utilise l'information mutuelle, concept que nous voyons au chapiter suivant en théorie de l'information.

Le résultat de la prochaine cellule est une liste de mots classés selon leur pouvoir discriminant du plus grand au plus petit. Le premier mot est donc celui qui permet de mieux classer les courriels des pourriels.

In [69]:
function mutualInformation(word::AbstractString, n₀::Int, n₁::Int, ham_wordcounts::Dict{String,Int64}, spam_wordcounts::Dict{String,Int64})
   
    p₁ = n₁/(n₀ + n₁)
    
    
    if haskey(ham_wordcounts, word)
        n₀₁ = ham_wordcounts[word]
    else
        n₀₁ = 0
    end
    
    if haskey(spam_wordcounts, word)
        n₁₁ = spam_wordcounts[word]
    else
        n₁₁ = 0
    end
    
    θ₀₁ = (1 + n₀₁) / (2 + n₀)
    θ₁₁ = (1 + n₁₁) / (2 + n₁)
    
    θ₁ = (1-p₁)*θ₀₁ + p₁*θ₁₁ 
    
    I_mat = [ (1-p₁)*(1-θ₀₁)*log( (1-θ₀₁)/(1-θ₁) ), (1-p₁)*θ₀₁*log( θ₀₁/θ₁ ),
        p₁*(1-θ₁₁)*log( (1-θ₁₁)/(1-θ₁) ), p₁*θ₁₁*log( θ₁₁/θ₁ )  ]
    
    I = sum(I_mat)
    
    return I
    
end

wordlist = unique(vcat(wordlisting.(vcat(ham_train, spam_train))...))
I = Float64[]

for word in wordlist
   push!(I, mutualInformation(word, length(ham_train), length(spam_train), ham_wordcounts, spam_wordcounts)) 
end

ind = sortperm(I, rev=true)

discriminantwords = wordlist[ind]

30530-element Vector{SubString{String}}:
 "vince"
 "enron"
 "cc"
 "kaminski"
 "ect"
 "pm"
 "hou"
 "am"
 "re"
 "thanks"
 "subject"
 "me"
 "forwarded"
 ⋮
 "plus"
 "key"
 "former"
 "moment"
 "consumers"
 "jun"
 "base"
 "near"
 "then"
 "such"
 "interests"
 "lines"

### (a) Écrivez une fonction généralisant votre fonction `filteremail()` pour une liste de mots

In [70]:
"""
    function filteremail(filename::AbstractString, words::Vector{<:AbstractString})

Filtre le message contenu dans le fichier `filename` en fonction de la liste de mots `words`.

### Détail
La valeur 1 est renvoyée si la probabilité prédictive que ce soit un courriel est supérieur à .5. 
Sinon, la valeur 0 est renvoyée
"""
function filteremail(filename::AbstractString, words::Vector{<:AbstractString})
    wordlist = wordlisting(filename)
    
    q₀ = log(p₀)
    q₁ = log(p₁)
    
    for word in words
        
        if word in wordlist
            q₀ += log(wordspamliness(word))
            q₁ += log(wordhamliness(word))
        else
            q₀ += log(1 - wordspamliness(word))
            q₁ += log(1 - wordhamliness(word))
        end
    end

    if q₁ > q₀
        return 1
    else
        return 0
    end
end

filteremail

### (c) Filtrez les messages de l'ensemble de validation avec votre fonction.

Utilisez les 10 mots les plus discriminants, ensuite les 100, les 1000, etc.

In [78]:
discriminant_words_10 = discriminantwords[1:10]

Z̃ = Int64[]

for filename in vcat(ham_valid, spam_valid)
    push!(Z̃, filteremail(filename, discriminant_words_10))
end
    
r = roc(Z, Z̃)

ROCNums{Int64}
  p = 1454
  n = 499
  tp = 1231
  tn = 491
  fp = 8
  fn = 223


In [80]:
discriminant_words_100 = discriminantwords[1:100]

Z̃ = Int64[]

for filename in vcat(ham_valid, spam_valid)
    push!(Z̃, filteremail(filename, discriminant_words_100))
end
    
r = roc(Z, Z̃)

ROCNums{Int64}
  p = 1454
  n = 499
  tp = 1294
  tn = 489
  fp = 10
  fn = 160


In [81]:
discriminant_words_1000 = discriminantwords[1:1000]

Z̃ = Int64[]

for filename in vcat(ham_valid, spam_valid)
    push!(Z̃, filteremail(filename, discriminant_words_1000))
end
    
r = roc(Z, Z̃)

ROCNums{Int64}
  p = 1454
  n = 499
  tp = 1378
  tn = 493
  fp = 6
  fn = 76
